In [1]:
import numpy as np
from numpy import loadtxt
from src.linear_algebra_resolution._history import History

In [2]:
gauss_partial_pivoting_b_path = 'test_matrix/gauss_partial_pivoting_b.csv'
gauss_partial_pivoting_b = loadtxt(gauss_partial_pivoting_b_path, delimiter=',')

gauss_partial_pivoting = gauss_partial_pivoting_b[:,:-1]
b = gauss_partial_pivoting_b[:,-1]

In [3]:
gauss_partial_pivoting

array([[5., 1., 0., 2., 1.],
       [0., 4., 0., 1., 2.],
       [1., 1., 4., 1., 1.],
       [0., 1., 2., 6., 0.],
       [0., 0., 1., 2., 4.]])

In [4]:
def partial_pivotation(x_b: np.ndarray, center_idx: int, history: History) -> tuple[np.ndarray, bool, History]:
	n = x_b.shape[0]
	x_b = x_b.copy()

	sucess = False
	for rown in range(center_idx + 1, n):

		test_value = x_b[rown, center_idx]

		if not np.isclose(test_value, 0):
			x_b = history.acum_operation_and_apply(
				history.permutation_rown,
				origin_rown_idx=center_idx,
				dest_rown_idx=rown,
				matrix=x_b,
				shape=n
			)
			sucess = True
			break

	return x_b, sucess, history

In [5]:
def total_pivotation(x_b: np.ndarray, center_idx: int, resp_arr_idx: np.ndarray, history: History) -> tuple[np.ndarray, np.ndarray, bool, History]:
	n = x_b.shape[0]
	x_b = x_b.copy()
	resp_arr_idx = resp_arr_idx.copy()

	sucess = False
	for col in range(center_idx + 1, n):

		test_value = x_b[center_idx, col]

		if not np.isclose(test_value, 0):

			x_b, resp_arr_idx = history.acum_operation_and_apply(
				history.permutation_col,
				origin_rown_idx=center_idx,
				dest_rown_idx=col,
				idx_arr=resp_arr_idx,
				matrix=x_b,
				shape=n
			)
			sucess = True
			break

	return x_b, resp_arr_idx, sucess, history

In [6]:
def process_matrix_correction(x_b: np.ndarray, center_idx: int, resp_arr_idx: np.ndarray, history: History) -> tuple[np.ndarray, np.ndarray, History]:

	x_b, sucess, history = partial_pivotation(x_b, center_idx, history)

	if not sucess:
		x_b, resp_arr_idx, sucess, history = total_pivotation(x_b, center_idx, resp_arr_idx, history)
		print('aq')

	if not sucess:
		err_txt = f"""\n{x_b} not exists valid value values to pivot number {center_idx}"""

		raise NameError(err_txt)

	return x_b, resp_arr_idx, history

In [7]:
def backward_substitution(x_b: np.ndarray) -> np.ndarray:
	n = x_b.shape[0]

	x = np.empty(n)

	x[n - 1] = x_b[n - 1, n] / x_b[n - 1, n - 1]

	for i in range(n - 2, -1, -1):
		x[i] = x_b[i, n]

		for j in range(i + 1, n):
			x[i] = x[i] - x_b[i, j] * x[j]

		x[i] = x[i] / x_b[i, i]

	return x

In [8]:
def gauss_forward_elimination(x_b: np.ndarray, history: History) -> tuple[np.ndarray, np.ndarray, History]:

	x_b = x_b.copy()
	n = x_b.shape[0]

	resp_order = np.arange(n, dtype=int)

	for i in range(n):

		if np.isclose(x_b[i, i], 0):
			x_b, resp_order, history = process_matrix_correction(x_b, i, resp_order, history)

		for j in range(i + 1, n):
			alpha = x_b[j, i] / x_b[i, i]

			x_b = history.acum_operation_and_apply(
				history.multiply_and_subtract_rown,
				origin_rown_idx=i,
				dest_rown_idx=j,
				matrix=x_b.copy(),
				shape=n,
				coef=alpha,
			)

	return x_b, resp_order, history

In [9]:
def gaussian_elimination(x: np.ndarray, b: np.ndarray) -> tuple[np.ndarray, History]:

	history = History(x)

	x_b = np.column_stack([x, b])

	x_forward, resp_order, history = gauss_forward_elimination(x_b, history)

	resp = backward_substitution(x_forward)

	return resp[resp_order], history

In [17]:
resp, history = gaussian_elimination(gauss_partial_pivoting, b)

In [18]:
b

array([1., 2., 3., 4., 5.])

In [19]:
gauss_partial_pivoting @ resp

array([1., 2., 3., 4., 5.])

In [20]:
history.base_matrix

array([[5., 1., 0., 2., 1.],
       [0., 4., 0., 1., 2.],
       [1., 1., 4., 1., 1.],
       [0., 1., 2., 6., 0.],
       [0., 0., 1., 2., 4.]])

In [21]:
m = history.apply_operations_matrix()

In [22]:
m

array([[ 5.        ,  1.        ,  0.        ,  2.        ,  1.        ],
       [ 0.        ,  4.        ,  0.        ,  1.        ,  2.        ],
       [ 0.        ,  0.        ,  4.        ,  0.4       ,  0.4       ],
       [ 0.        ,  0.        ,  0.        ,  5.55      , -0.7       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  4.13963964]])

In [23]:
history.operations_matrix

[array([[ 1. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  1. ,  0. ,  0. ,  0. ],
        [-0.2,  0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  1. ]]),
 array([[ 1. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  1. ,  0. ,  0. ,  0. ],
        [ 0. , -0.2,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  1. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  1. ]]),
 array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  , -0.25,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ]]),
 array([[ 1. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  1. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. , -0.5,  1. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  1. ]]),
 array([[ 1.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  1.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1